In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pandas as pd
import numpy as np
from datasurfer.lib_objects.pdf_object import PDFPagesObject
from datasurfer.lib_objects.docx_object import DOCXObject
from datasurfer.datautils import xml_valid_df

In [17]:
pobj = PDFPagesObject(r'D:\02_Translation\01_Künstliche_Intelligenz_für_Ingenieure\KII.pdf', range(175-1, 208))
fdst = 'KII_Chapter_06_Original.xlsx'

In [18]:
annos = ['© Walter de Gruyter GmbH Berlin Boston. All rights reserved. \n',
 'This work may not be translated, copied or made publicly accessible in whole or part without the written permission of the publisher  \n',
 '(Walter De Gruyter GmbH, Genthiner Straße 13, 10785 Berlin, Germany). \n',
 'Vertraulich\n']

In [19]:
df = pobj.df[~pd.concat([pobj.df.text == s for s in annos], axis=1).any(axis=1).values]

df

,x0,y0,x1,y1,text,index,type,page_num
0,70.944000,37.439972,73.439857,48.479969,\n,0,0,174
1,70.944000,795.239929,73.439857,806.279968,\n,1,0,174
2,70.944000,50.879974,122.995857,102.260002,\n \n \n \n \n,2,0,174
6,48.168003,444.284119,59.208000,446.779968,\n,6,0,174
7,61.728004,822.336121,72.768005,824.831970,\n,7,0,174
...,...,...,...,...,...,...,...,...
758,70.944000,37.439972,73.439857,48.479969,\n,0,0,207
759,70.944000,795.239929,73.439857,806.279968,\n,1,0,207
760,70.944000,50.879974,122.995857,102.260002,\n \n \n \n \n,2,0,207
764,48.168003,444.284119,59.208000,446.779968,\n,6,0,207


In [20]:

df = df[df.text.str.strip().str.len()>0].reset_index(drop=True)

df['rel_pagenum'] = df.page_num - df.page_num.min()

df

,x0,y0,x1,y1,text,index,type,page_num,rel_pagenum
0,426.054108,213.080688,481.844788,312.706879,6\n,8,0,174,0
1,169.133698,315.166351,482.095490,342.317535,Funktionale Programmierung in LISP\n,9,0,174,0
2,121.341492,384.741638,466.919708,433.578339,Bei der Programmiersprache LISP werden alle Ve...,10,0,174,0
3,104.933502,478.696014,368.864014,492.420380,6.1 Einführung in die funktionale Programmieru...,11,0,174,0
4,104.933502,504.548676,218.153030,515.986206,6.1.1 Grundidee von LISP\n,12,0,174,0
...,...,...,...,...,...,...,...,...,...
450,104.213509,195.728790,306.332886,208.820175,und dem im DANN-Teil angegebenen Seiteneffekt\n,11,0,206,32
451,139.253387,217.139267,205.208450,240.163727,WASSERDRUCK\n⇒NIEDRIG\n,12,0,206,32
452,104.920883,247.556274,481.970398,272.643127,Hier wird die REGEL_1 offensichtlich als Progr...,13,0,206,32
453,104.933502,305.295715,196.389252,319.020081,Literaturhinweise\n,14,0,206,32


In [21]:
df[df.rel_pagenum == 1]

,x0,y0,x1,y1,text,index,type,page_num,rel_pagenum
6,113.333603,117.544724,490.550964,129.326050,152\n6 Funktionale Programmierung in LISP\n,8,0,175,1
7,112.839134,144.020844,492.219543,252.752426,"Aussage, dass die Klasse der berechenbaren Fun...",9,0,175,1
8,113.329185,262.825012,490.767822,287.792297,Funktionale Sprachen. LISP ist eine funktional...,10,0,175,1
9,203.564087,296.659363,490.568237,309.750763,(OPERATOR OPERAND_1 OPERAND_2 . . .).\n(6.1)\n,11,0,175,1
10,113.319252,318.617828,490.528229,367.576019,Das erste Element gibt den Namen der Funktion ...,12,0,175,1
11,139.056610,464.226196,460.153564,486.382599,    \n   \n   \n  ...,13,0,175,1
12,173.302704,408.090088,412.707886,418.038666,        \n   ...,14,0,175,1
13,217.613708,498.425537,381.314941,510.206848,Abb. 6.1: Funktion als Zuordnungsvorschrift\n,15,0,175,1
14,113.313492,529.461548,490.738007,662.162109,Der Begriff „Funktion“ wird in gleicher Weise ...,16,0,175,1


In [22]:

def remove_title_line(df):
    count = 0
    for i, (text, num) in df[['text', 'rel_pagenum']].iterrows():  
        if num > count:
            count += 1       
        else:        
            yield([text, num])
        
out0 = list(remove_title_line(df))



In [23]:
def split_paragraph(text, threshold=0.8):
    
    lst_text = np.array([txt for txt in text.split('\n') if txt.strip()])
    larr = np.array([len(txt) for txt in lst_text])
    isnewline = larr < (larr.max() * threshold)
    
    out = ''
    for isnl, txt in zip(isnewline, lst_text):
        if isnl:
            out += txt
            yield out
            out = ''
        else:
            out += txt.rstrip('-')
            
    if len(out) > 0:      
        yield out

    


In [24]:

def merge_paragraphs(out0):
    count = 0 

    for text, num in out0:
        for txt in split_paragraph(text):
            yield [txt, num]
            
out1 = list(merge_paragraphs(out0))

In [25]:

def merge_paragraphs(out1):
    
    iterator = iter(out1)
    line0, num0 = next(iterator)
    while True:
        try:       
            line1, num1 = next(iterator)
        except StopIteration:
            yield line0, num0
            break
        if num1 > num0:
        
            if line0.strip()[-1] not in '.!?:;':
                line = line0 + ' ' + line1
                yield line, num0
                try:
                    line0, num0 = next(iterator)
                except StopIteration:
                    yield line0, num0
                    break
            else:
                yield line0, num0
                line0 = line1
                num0 = num1            
        else:
            yield line0, num0   
            line0 = line1
            num0 = num1
            
out2 = list(merge_paragraphs(out1))

In [26]:
dfout = pd.DataFrame(out2, columns=['Original', 'Page'])
dfout['Translation'] =  None
dfout['Review'] = 4 

dfout = xml_valid_df(dfout[['Page', 'Original', 'Translation', 'Review']])
dfout.Page = dfout.Page.astype('int') + df.page_num.min() + 1

dfout = dfout[~dfout['Original'].str.isnumeric()]
dfout = dfout[~(dfout['Original'].str.strip().str.len()<2)]
dfout.reset_index(drop=True, inplace=True)

In [27]:
dfout.head()

,Page,Original,Translation,Review
0,175,Funktionale Programmierung in LISP,None,4
1,175,Bei der Programmiersprache LISP werden alle Ve...,None,4
2,175,6.1 Einführung in die funktionale Programmierung,None,4
3,175,6.1.1 Grundidee von LISP,None,4
4,175,LISP (List Processing Language) ist eine Progr...,None,4


In [28]:
dfout.to_excel(fdst, index=False)